In [1]:
from Utils import normalize_meshId,preprocess_text
from pandas import read_csv
from Data_retrieval import load_dataset
from Evalutation import graphType, gridsearch, build_graph_from_config, plot_hist

from itertools import product

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrea\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [2]:
df_texts, df_entities = load_dataset()

df_entities = df_entities[~df_entities['prob'].isna()]
df_entities = df_entities[df_entities['prob'] > 0.8]
df_entities = df_entities[['id', 'pmid', 'mention', 'obj', 'span_begin', 'span_end']]

## Test set creation

To test the knowledge graph is used the SCMFDD dataset, there are three kinds of files: diseases, drugs and diseases-drugs associations.

The dataframe `diseases-drug` associations contains:
- drug_id: id of drug
- drug_name: plain name of drug
- disease_id: id of disease
- disease_name: plain name of disease

In [12]:
SCMFF_DIR = 'dataset/SCMFDD/SCMFDD-L/'

scmff_dda = read_csv(f'{SCMFF_DIR}drug-disease association.csv')
scmff_drugs = read_csv(f'{SCMFF_DIR}drug.csv')
scmff_diseases = read_csv(f'{SCMFF_DIR}disease.csv')

Add column `source` that is `true` for each disease of `df_entities` that is also inside the test set.

In [13]:
scmff_diseases_labels = scmff_diseases['disease_id'].unique()
union_diseases = list(filter(lambda x: normalize_meshId(x) in scmff_diseases_labels,
                             df_entities[df_entities['obj'] == 'disease']['id'].unique()))

df_entities['source'] = (df_entities['obj'] == 'disease') & (df_entities['id'].isin(union_diseases))

sum(df_entities['source'])

64555

In [14]:
scmff_dda['drug_id'] = scmff_dda['drug_id'].apply(normalize_meshId)
scmff_dda['disease_id'] = scmff_dda['disease_id'].apply(normalize_meshId)

normalized_union_diseases = map(normalize_meshId, union_diseases)
scmff_dda = scmff_dda[scmff_dda['disease_id'].isin(normalized_union_diseases)]

In [15]:
test_set = {
    (
        v['drug_id'],
        v['disease_id']
    ): True for v in scmff_dda[['drug_id', 'disease_id']].to_dict(orient='records')
}

In [ ]:
id_to_wuid = {id: f'unique_{i}' for i, id in enumerate(df_entities['id'].unique())}
texts = df_texts.progress_apply(preprocess_text(df_entities,id_to_wuid), axis=1).tolist()

In [16]:
coc_configs = dict(
    k=list(range(5, 41, 5))
)

w2v_configs = dict(
    min_count=[2, 5],
    vector_size=[100, 500],
    window=[5, 10],
    sg=[False, True],
    learning_rate=[0.1],
    epochs=[100],
)

configs = [
    *[(graphType.COOCCURRENCES, dict(zip(coc_configs, t))) for t in product(*coc_configs.values())],
    *[(graphType.WORD2VEC, dict(zip(w2v_configs, t))) for t in product(*w2v_configs.values())],
]

ranking = gridsearch(configs=configs,
                     ts_set=test_set,
                     texts=texts,
                     df_entities=df_entities,
                     firsts_selected_diseases=100,
                     validity=10)

In [ ]:
for precision, config in ranking:
    print(precision, config)

In [ ]:
precision, config = ranking[0]
graph = build_graph_from_config(config, df_entities, test_set)

In [ ]:
plot_hist(graph,10)

In [ ]:
plot_hist(graph,30)

In [ ]:
plot_hist(graph,100)